In [1]:
import numpy as np
from numpy.linalg import norm
from sklearn import datasets,neighbors
from sklearn.metrics.pairwise import euclidean_distances,pairwise_distances_argmin
from sklearn.model_selection import LeaveOneOut, cross_val_score
from statistics import mode
iris=datasets.load_iris()
X=iris.data
y=iris.target


In [2]:
from numpy import append, vstack
from pyparsing import disable_diag


def PPV(X,y):
    dist_XX=euclidean_distances(X,X)
    d=np.zeros((len(dist_XX),len(dist_XX)-1))
    
    for i in range(len(dist_XX)):
        d[i]=np.delete(dist_XX[i],i)
    
    
    
    ppd=d.argmin(axis=0)
    print(ppd)
    labld=np.zeros((150,))
    for i in range(len(ppd)):
       labld[i]=y[ppd[i]]
    

    
    
    predit=np.concatenate((X,labld.reshape(150,1)),axis=1)
    e=list(y!=labld).count(True)/150
    
    return e
PPV(X,y)




[ 17  12  47   0  37   2   0  39  38  34  48   1  34  38  33  10   0  40
   4  21  19  46   7  26   9  34   0  27  39  29  34  36  32   9  45  49
   4   8   7  17  49  38  26   5   1  19   3  10   7  98  52  50  86  89
  58  51  85  93  75  89  93  96  54  91  58  75  84  92  87  80 138  97
  63  91  65  58  52  63  91  69  80  67  92  66  56  52  68  61  95  94
  63  82  57  90  88  95  99  57  96 138  83 142 125 116 132 122 105 130
 128  77 147 147 139 142 110 103 137 131 122 112 143 105  72 126 143 123
 126 138 132 107 117 128  83 146 130 115 116 127 112 120 144 101 120 140
 141 123 110 136 127]


0.05333333333333334

In [3]:
loo=LeaveOneOut()
knn=neighbors.KNeighborsClassifier(n_neighbors=1)
score=[]
for train, test in loo.split(X,y):
    knn.fit(X[train],y[train])
    scores=knn.score(X[test],y[test])
    
    score.append(scores)
s=0
for e in score:
    s +=e
scor=s/150
print(scor)



0.96


In [23]:
def PPV(X,y,k):
    dist_XX=euclidean_distances(X,X)
    d=np.zeros((len(dist_XX),len(dist_XX)-1))
    
    for i in range(len(dist_XX)):
        d[i]=np.delete(dist_XX[i],i)
    
    predict=np.zeros((X.shape[0],))
    for i in range(X.shape[0]):
        d_i=d[i,:]
        idx_sort=np.argsort(d_i)
        
        neigh=idx_sort[0:k]
        predict[i]=mode(y[neigh])
    return predict

PPV(X,y,1)


array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 2., 1., 2., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 2., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 2., 2.,
       2., 2., 2., 2., 1., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2.,
       1., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 1., 2., 2.,
       2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2.])

In [57]:
def CNB(X,y):
    #barycentres des classes
    #probabilité d'appartenance à priori de chaque classe
    p=len(np.unique(y))
    proba_class=np.zeros((p,))
    class_mean=np.zeros((p,X.shape[1]))
    for i,c in enumerate(np.unique(y)):
        x=X[y==c]
        proba_class[i]=len(x)/len(X)
        class_mean[i]=x.mean(axis=0)
    #probalité d'une donnée d'appartenir à une classe
    proba_cond=np.zeros((X.shape[0],p))
    d_ij=np.zeros((X.shape[0],p))
    one=np.ones((X.shape[0],p))
    for i,d in enumerate(X):
        for j,w in enumerate(class_mean):
            n=norm(d-w,ord=2)
            d_ij[i,j]=n 
        d_ij[i,:]=d_ij[i,:]/d_ij[i,:].sum()
    proba_cond=one-d_ij
    # prédire la classe qui maximise la probabilité d'appartenance
    predit=np.zeros((X.shape[0],))
    pred_i=np.zeros((X.shape[0],))
    for i,d in enumerate(proba_cond):
        pred_i[i]=np.argmax(proba_cond[i]*proba_class)
    #calculer l'erreur de prédiction 
    e=list(pred_i != y).count(True)/len(X)
    
    return e
print(CNB(X,y))

    
    
    


0.07333333333333333


In [53]:
from sklearn.naive_bayes import GaussianNB
gnb=GaussianNB()
score=[]
for train, test in loo.split(X,y):
    gnb.fit(X[train],y[train])
    scores=gnb.score(X[test],y[test])
    
    score.append(scores)
s=0
for e in score:
    s +=e
scor=s/150
print(scor)

0.9533333333333334
